# 第6回 その1: 線形判別分析を用いた教師ありクラスタリング
第5回で紹介した線形判別分析を使って教師有りクラスタリングを行います。  


## ステップ0: Google Driveのマウントと作業フォルダへの移動  
Google Drive に配置したデータを読み込むための準備です。  
詳細については第二回の 02_01_graph.ipynb を参照してください。  

ここでは"マイドライブ/情報管理/06"を作業フォルダとします。 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# フォルダの移動には"%cd"を使用します。
# 作業フォルダへ移動
%cd /content/drive/'My Drive'/情報管理/06/
# 現在のフォルダの中身を表示
%ls

`2class_data.csv`というデータが表示されていることを確認してください。

## ステップ1: データの読み込みと標準化
まずは必要ライブラリをインポートします。

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

`2class_data.csv` を読み込みます。  
このデータx1とx2の二次元データで，クラス0とクラス1の2種類のクラスのどちらかに属しています。

In [ ]:
# pandas の関数 read_csv を用いた csvファイル読み込み
csv_data = pd.read_csv('2class_data.csv', encoding='SHIFT-JIS')

# データの前半部(.headで取得できる)のみ表示
display(csv_data.head())

# x1とx2のデータを抽出し，numpy用データ(ndarray型) に変換する。
X = csv_data.loc[:, ['x1','x2']].to_numpy()

# クラス番号を抽出し，numpy用データ(ndarray型) に変換する。
Y = csv_data.loc[:,'class'].to_numpy()

# データのサンプル数と次元数を得る。
(num_samples, num_dimensions) = np.shape(X)
print('Nunber of samples: ' + str(num_samples))
print('Number of dimensions: ' + str(num_dimensions))

# クラス数を得る。
num_classes = int(np.max(Y) + 1)
print('Number of classes: ' + str(num_classes))

標準化を行い，データをプロットします。

In [ ]:
# データを標準化する。
X = (X - np.mean(X, axis=0)) / np.std(X, axis=0)

# データをプロット
plt.figure(figsize=(7,7))
plt.scatter(X[Y==0,0], X[Y==0,1], c='b', label='class0')
plt.scatter(X[Y==1,0], X[Y==1,1], c='r', label='class1')
plt.xlabel('x1')
plt.ylabel('x2')
plt.legend()
plt.xlim([-3, 3])
plt.ylim([-3, 3])
plt.show()

## ステップ2: 線形判別分析  
線形判別分析を行います。  
以下のソースコードは第5回の`05_02_lda.ipynb`を流用したものです。  

まずはクラス内分散-クラス間分散比の行列を計算します。

In [ ]:
# クラス全体の平均
m_all = np.mean(X, axis=0)

# クラス内分散共分散
cov_inner = np.zeros((num_dimensions, num_dimensions))
for n in range(num_classes):
  # クラス n に該当するデータ X_tmp
  X_tmp = X[Y==n]
  # クラス内平均
  m = np.mean(X_tmp, axis=0)
  cov_inner += np.dot((X_tmp-m).T, (X_tmp-m))

# クラス間分散
cov_intra = np.zeros((num_dimensions, num_dimensions))
for n in range(num_classes):
  # クラス n に該当するデータ X_tmp
  X_tmp = X[Y==n]
  # クラス n に該当するデータのサンプル数を得る
  num_samples_n, num_dimensions = np.shape(X_tmp)
  # クラス内平均
  m = np.mean(X_tmp, axis=0)

  # クラス間分散を計算
  m_sub = m - m_all
  m_sub = np.reshape(m_sub, (-1,1))
  cov_intra += num_samples_n * np.dot(m_sub, m_sub.T)

# クラス内分散-クラス間分散比行列
J = np.dot(np.linalg.inv(cov_inner), cov_intra)

続いてクラス内分散-クラス間分散比行列を固有値分解し，固有値が最も大きい固有ベクトル（第一固有ベクトル）を取り出します。

In [ ]:
# 固有値 eig_val と 固有ベクトル eig_vec を計算
eig_val, eig_vec =np.linalg.eig(J)
# データによっては固有値分解結果が複素数になる場合があるため，実部のみ取得
eig_val = np.real(eig_val)
eig_vec = np.real(eig_vec)

# 固有値が最大となる固有ベクトルを取り出す
max_id = np.argmax(eig_val)
first_eig_vec = eig_vec[:, max_id]

print(first_eig_vec)

第一固有ベクトルへ射影（内積を計算）し，その結果を散布図で描画します。  
（ただし，散布図を書くにははx-yの2次元データが必要なのに対して，射影結果は1次元のスカラー値です。なので，y軸の値は常に0を入れることで，散布図を書いています。）

In [ ]:
X_lda = np.dot(X,first_eig_vec)

plt.figure(figsize=(7,5))
# x軸にはX_ldaの値を，y軸には0を入れて散布図を描画
plt.scatter(X_lda[Y==0], np.zeros(np.size(X_lda[Y==0])), color='b', label='class 0')
plt.scatter(X_lda[Y==1], np.zeros(np.size(X_lda[Y==1])), color='r', label='class 1')
plt.xlabel('projection axis')
plt.legend()
plt.show()

第一固有ベクトルへの射影により，クラス0とクラス1がうまく分離出来ていることが分かります。  
実際に，X_ldaの値が正ならクラス1，負ならクラス0として2クラス分類を行い，分類正解率を測ってみます。  

In [ ]:
# 全ての要素がゼロのベクトルを用意
Y_pred = np.zeros(num_samples, dtype=np.int64)
# X_ldaが正なら Y_pradを1にする。
Y_pred[X_lda > 0] = 1

# 正解率を計算
accuracy = 0
for n in range(num_samples):
  if Y_pred[n] == Y[n]:
    # 分類結果と正解ラベルが一致していれば，正解数のカウントを1増やす
    accuracy += 1
accuracy = 100.0 * accuracy / num_samples

print('Accuracy = %.3f' % (accuracy))

分類正解率が100%であることが分かりました。

### クラスの境界を可視化する。
上の処理では，第一固有ベクトルと内積を計算し，計算結果が正ならクラス1，0以下ならクラス0として2クラス分離していました。  
よって，第一固有ベクトルを${\bf w} = [w_1, w_2]$，データを${\bf x} = [x_1, x_2]$とすると，  
${\bf w}{\bf x}^{T} = w_1x_1 + w_2x_2 = 0$  
がこの2クラスを分離する境界線ということになります。  
この式を変形して  
$x_2 = - w_1x_1 / w_2$  
とすれば，この境界線を図示することができます。

In [ ]:
# 境界線の作成
x1 = np.linspace(-3,3) # -3から3へ一定間隔で増える等差数列
x2 = -1.0 * first_eig_vec[0] * x1 / first_eig_vec[1]

# プロット
plt.figure(figsize=(7,7))
plt.plot(x1, x2)
plt.scatter(X[Y==0,0], X[Y==0,1], c='b', label='class0')
plt.scatter(X[Y==1,0], X[Y==1,1], c='r', label='class1')
plt.xlabel('x1')
plt.ylabel('x2')
plt.legend()
plt.xlim([-3, 3])
plt.ylim([-3, 3])
plt.show()

これが線形判別分析によって導出された，2クラスの境界線です。